In [91]:
pip install catboost

In [92]:
pip install catboost optuna

In [93]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [94]:
import os

print(os.listdir('./gdrive/MyDrive/SHIFT_ML/public_dat'))

['train.csv']


In [95]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from xgboost import XGBRegressor
import xgboost as xgb

import optuna

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error
import os

from catboost import CatBoostRegressor
print(os.listdir('./gdrive/MyDrive/SHIFT_ML/public_dat'))

['train.csv']


In [96]:
df = pd.read_csv('./gdrive/MyDrive/SHIFT_ML/public_dat/train.csv', index_col=0)
df

,full_sq,floor,build_year,num_room,kitch_sq,state,product_type,raion_popul,indust_part,sport_objects_raion,...,cafe_count_1000_price_high,cafe_sum_1500_min_price_avg,green_part_2000,cafe_sum_2000_min_price_avg,mosque_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,year,price_doc
0,43,24.0,2014.0,1.0,10.0,1.0,1,8.294300,0.007122,0,...,0,6.907755,3.710396,6.725430,0,1.803359,6.616560,0,2014,5533460
1,66,3.0,1979.0,3.0,10.0,3.0,0,11.314231,0.232205,4,...,0,6.626122,2.764431,6.479615,1,2.631169,6.469948,1,2013,9900000
2,55,5.0,1979.0,2.0,6.0,2.0,0,11.718817,0.000170,5,...,0,6.368753,3.511844,6.192138,0,1.363537,6.462171,0,2012,1990000
3,45,7.0,1970.0,2.0,6.0,2.0,0,11.331032,0.428826,4,...,0,6.753601,1.814825,6.551080,0,2.732418,6.458229,0,2014,6500000
4,43,4.0,1968.0,2.0,6.0,2.0,0,11.538769,0.136330,6,...,0,6.636603,2.257588,6.646507,1,2.020222,6.634778,2,2014,8500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21324,56,19.0,1979.0,2.0,1.0,1.0,1,11.655744,0.049637,6,...,0,6.492240,2.711378,6.816188,0,3.297317,6.536155,0,2014,10159562
21325,42,5.0,1977.0,1.0,8.0,3.0,0,11.718817,0.000170,5,...,0,6.502295,3.802208,6.291569,0,1.442202,6.487684,0,2014,5100000
21326,32,3.0,1979.0,2.0,6.0,2.0,0,11.719940,0.000000,4,...,0,6.572814,3.062456,6.583520,1,2.733718,6.573904,2,2011,4800000
21327,43,4.0,1969.0,2.0,5.0,2.0,0,11.331032,0.428826,4,...,0,6.586172,2.408745,6.563982,0,3.034953,6.476311,0,2013,6300000


In [97]:
X = df.iloc[:, :-1]  # Все столбцы кроме последнего
y = df.iloc[:, -1]   # Только последний столбец

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 1000, 6000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 3, 7),
    }

    model = CatBoostRegressor(**params, silent=True)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=35)
print('Best hyperparameters:', study.best_params)
print('Best RMSE:', study.best_value)

[I 2024-07-11 11:01:17,972] A new study created in memory with name: no-name-c5f87336-e316-4d8f-ad2c-415c4caf1ac2
[I 2024-07-11 11:01:50,290] Trial 0 finished with value: 2583172.0414799424 and parameters: {'iterations': 2632, 'learning_rate': 0.05323374251372339, 'depth': 4}. Best is trial 0 with value: 2583172.0414799424.


Best hyperparameters: {'iterations': 2632, 'learning_rate': 0.05323374251372339, 'depth': 4}
Best RMSE: 2583172.0414799424


In [98]:
X = df.iloc[:, :-1]  # Все столбцы кроме последнего
y = df.iloc[:, -1]   # Только последний столбец

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стадартизация признаков - в нашем случаем результат на тренировочной выборке становится лучше, а на тестовой хуже, как следствие результат на codalab становится хуже
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

X_train=X
y_train=y

model_xgbr = CatBoostRegressor(iterations = 2000, depth = 6, learning_rate = 0.025)
model_xgbr.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model_xgbr.predict(X_test)

# Оценка модели
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
# Вывод результатов
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.5f}")
print()

0:	learn: 4643673.4522597	total: 27.8ms	remaining: 55.5s
1:	learn: 4589225.5848419	total: 59.8ms	remaining: 59.7s
2:	learn: 4533058.0677083	total: 87ms	remaining: 57.9s
3:	learn: 4479010.8341195	total: 114ms	remaining: 56.8s
4:	learn: 4426305.6312888	total: 146ms	remaining: 58.1s
5:	learn: 4375416.5517468	total: 180ms	remaining: 60s
6:	learn: 4324013.3205935	total: 212ms	remaining: 1m
7:	learn: 4274698.6196332	total: 234ms	remaining: 58.2s
8:	learn: 4226886.8038031	total: 255ms	remaining: 56.4s
9:	learn: 4182772.6076044	total: 290ms	remaining: 57.7s
10:	learn: 4140036.4552480	total: 324ms	remaining: 58.7s
11:	learn: 4097238.5989167	total: 359ms	remaining: 59.5s
12:	learn: 4055266.6316440	total: 395ms	remaining: 1m
13:	learn: 4016280.1352239	total: 433ms	remaining: 1m 1s
14:	learn: 3978010.5562094	total: 457ms	remaining: 1m
15:	learn: 3940550.2887654	total: 483ms	remaining: 59.9s
16:	learn: 3904331.8644625	total: 518ms	remaining: 1m
17:	learn: 3868231.1150524	total: 554ms	remaining: 1m 

In [99]:
submission = pd.read_csv('./gdrive/MyDrive/SHIFT_ML/starting_k/submission.csv', index_col=0)

In [100]:
test_df = pd.read_csv('./gdrive/MyDrive/SHIFT_ML/starting_k/test.csv', index_col=0)

test_preds = model_xgbr.predict(test_df[X.columns])


submission['price_doc'] = test_preds
print(len(test_preds))
print(submission.shape)
submission.to_csv('./gdrive/MyDrive/SHIFT_ML/submission.csv')

9142
(9142, 1)


In [101]:
test_preds

array([4702925.44777258, 3659875.67609976, 5536111.53433305, ...,
       5804310.11876492, 6958451.6710594 , 5979726.63861917])